<a href="https://colab.research.google.com/github/SatadruMukherjee/Data-Preprocessing-Models/blob/main/AWS%20Pinpoint%20using%20Boto3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

In [19]:
import json
import base64
import boto3
import time
import os

In [20]:
pinpoint_client= boto3.client('pinpoint',  aws_access_key_id='',
aws_secret_access_key='', region_name='us-east-1')
application_id=''

In [21]:
def segment_creator(application_id,s3url,pinpint_role_arn):
    response = pinpoint_client.create_import_job(
              ApplicationId=application_id,
              ImportJobRequest={
                  'DefineSegment': True,
                  'Format':'CSV',
                  'RoleArn': pinpint_role_arn,
                  'S3Url': s3url,
                  'SegmentName': 'primary_segment'
              }
            )
    return response

In [22]:
def campaign_creator(application_id,Body,FromAddress,HtmlBody,Title,campaign_name,segment_id):
    response=pinpoint_client.create_campaign(
                ApplicationId=application_id,
                WriteCampaignRequest={
                    "MessageConfiguration": {
                        "EmailMessage": {
                          "Body":Body,
                          "FromAddress": FromAddress,
                          "HtmlBody": HtmlBody,
                        "Title": Title
                        }},
                    "Name": campaign_name,
                    "Schedule": {
                        "Frequency": "ONCE",
                        "StartTime": "IMMEDIATE",
                        "Timezone": "UTC"
                    },
                    "SegmentId": segment_id
                    })
    return response

In [23]:
def main(Subject,HtmlBody,s3url,pinpint_role_arn):
    
    response_import_job=segment_creator(application_id,s3url,pinpint_role_arn)
    
    segment_id=response_import_job['ImportJobResponse']['Definition']['SegmentId']
    print("Segment ID for this Campaign : {}".format(segment_id))
    
    job_id=response_import_job['ImportJobResponse']['Id']
    print("Job ID for this Campaign : {}".format(job_id))
    
    temp=0
    job_status=''
    while True:
        response_info_about_job = pinpoint_client.get_import_job(
        ApplicationId=application_id,
        JobId=job_id)
        job_status=response_info_about_job['ImportJobResponse']['JobStatus']
        print("Job status in Poll  {} is {}".format(temp,job_status))
        if(job_status=='COMPLETED'):
            break
        elif(job_status=='FAILING' or job_status=='FAILED'):
            break
        time.sleep(10)
        temp=temp+1;
        
    if(job_status=='COMPLETED'):
        campaign_name="Campaign for Testing from Google Colab"
        Title=Subject
        FromAddress=""
        Body="Just for demo testing"
        response_of_campaign=campaign_creator(application_id,Body,FromAddress,HtmlBody,Title,campaign_name,segment_id)
        print(response_of_campaign)
        capaign_id=response_of_campaign['CampaignResponse']['Id']
        return "Campaign ID : {}".format(capaign_id)
    elif(job_status=='FAILING' or job_status=='FAILED'):
        return "Import Job Failed for Segment Creation"

In [24]:
HtmlBody=""
with open('/content/Axis Bank.html','r') as f:
  HtmlBody=f.read()

In [ ]:
pinpint_role_arn=""
Subject="Hello {{User.UserAttributes.Username}} , don't worry about this Promotional mail"
s3url=""
op=main(Subject,HtmlBody,s3url,pinpint_role_arn)